In [3]:
# ===================================
# !!! SET UP PATH AND MERGE FILES !!!
# ===================================

import csv
import numpy as np
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
import os

Nchunk = 20
trials = 1e4
directory = '/home/ambra/Desktop/cluster-morgana/archive_tests/tesi_01_preliminary/'
nfolders = ['tesi_preliminary_nominal_noEBL/', 'tesi_preliminary_nominal_EBL/']
dfolders = ['tesi_preliminary_degraded_noEBL/', 'tesi_preliminary_degraded_EBL/']
folders = nfolders + dfolders

print(dfolders, nfolders, folders)

['tesi_preliminary_degraded_noEBL/', 'tesi_preliminary_degraded_EBL/'] ['tesi_preliminary_nominal_noEBL/', 'tesi_preliminary_nominal_EBL/'] ['tesi_preliminary_nominal_noEBL/', 'tesi_preliminary_nominal_EBL/', 'tesi_preliminary_degraded_noEBL/', 'tesi_preliminary_degraded_EBL/']


In [4]:
for i, f in enumerate(folders):
    os.system('mkdir %s/png' %(directory+f))
    path = directory + f + 'run0406/run0406_ID000126/csv/'

    texp = [1,5,10,100]
    sigma = [5]
    chunk = [i+1 for i in range(Nchunk)]

    csvName = [[]*i for i in range(len(texp))]
    for i in range(len(chunk)) :  
        for j in range(len(texp)) :
            csvName[j].append('run0406_%ds_chunk%02d.csv' % (texp[j], chunk[i]))
            
    # merge files ---!
    csvMerged = []
    for j in range(len(texp)) :
        csvMerged.append('run0406_%ds.csv' %texp[j])

        fout = open(path+csvMerged[j],'w+')
        # first file ---!
        for line in open(path+csvName[j][0]):
            fout.write(line)
        # remaining files ---!    
        for i in range(len(chunk)-1):
            f = open(path+csvName[j][i+1])
            next(f) # skip the header ---!
            for line in f:
                 fout.write(line)
            f.close() 
        fout.close()

    print(csvMerged)
print('done')

['run0406_1s.csv', 'run0406_5s.csv', 'run0406_10s.csv', 'run0406_100s.csv']
['run0406_1s.csv', 'run0406_5s.csv', 'run0406_10s.csv', 'run0406_100s.csv']
['run0406_1s.csv', 'run0406_5s.csv', 'run0406_10s.csv', 'run0406_100s.csv']
['run0406_1s.csv', 'run0406_5s.csv', 'run0406_10s.csv', 'run0406_100s.csv']
done


In [15]:
# reformat files ---!
#for f in csvMerged:
#    with open(path+f, 'r') as inp, open(path+f.replace('.csv', '_mod.csv'), 'w') as out:
#        writer = csv.writer(out)
#        for row in csv.reader(inp):
#            if len(row) == 6 and ('ID' in row[0] or '#' in row[0]):
#                writer.writerow(row)


#for index, f in enumerate(csvMerged):
#    csvMerged[index] = f.replace('.csv', '_mod.csv')
#print(csvMerged)

In [7]:
# data params ---!
threshold = 0.5
xlim=0.2
# initialize SphDistance calc ---!
trueRA = 33.057
trueDEC = -51.841
true_coord = SkyCoord(ra = trueRA*u.deg, dec = trueDEC*u.deg, frame='fk5')

In [14]:
# ------------------------------------------ plots ---!
def plottage(texp, ra, dec, pos, std_ra, std_dec, std_pos, trueRA, trueDEC, png_path):

    c = ['darkgreen', 'red', 'navy', 'gold']
    alpha = 0.3
    hist=True
    
    # conf intervals params ---!
    nstd = [1,2,3,5]
    conf_probs = [0.6827, 0.9545, 0.9973, 0.99994]
    # img params ---!
    fontsize=12
    CI_colors = ['k', 'r', 'g', 'w']

    # DETECTION 1D ---!
    fig, ax = hist1d_gauss(ra, mean=None, loc=trueRA, threshold=threshold, nbin=Dnbin, width=Dwidth,
                           fontsize=fontsize, color=c, alpha=alpha, hist=hist,
                           title='detected RA distribution', 
                           xlabel='RA (deg)', ylabel='counts density',
                           filename=png_path+'run0406_detRA_h%s.png' %str(hist))   
    plt.show()
    plt.close()

    fig, ax = hist1d_gauss(dec, mean=None, loc=trueDEC, nbin=Dnbin, width=Dwidth,
                           threshold=threshold, fontsize=fontsize, color=c, alpha=alpha,
                           title='detected DEC distribution', hist=hist, 
                           xlabel='DEC (deg)', ylabel='counts density',
                           filename=png_path+'run0406_detDEC_h%s.png' %str(hist)) 
    plt.show()
    plt.close()

    fig, ax = hist1d_rayleigh(pos, mean=None, rayleigh_prms={'loc':0, 'scale':None}, 
                              threshold=threshold, nbin=Dnbin, width=Dwidth, fontsize=fontsize, 
                              color=c, alpha=alpha, hist=hist,
                              title='detected $\\Delta \\Theta$ distribution',
                              xlabel='$\\Delta \\Theta$ (deg)', ylabel='counts density',
                              filename=png_path+'run0406_detPOS_h%s.png' %str(hist))
    plt.show()
    plt.close()

    
    
print('\n\nok')



ok


In [21]:
# =============================
# !!! ALL STATISTICAL PLOTS !!!
# =============================

import matplotlib.pyplot as plt
import seaborn as sns
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy import stats
from scipy.stats import rayleigh, norm
from matplotlib.lines import Line2D
from matplotlib.patches import Ellipse, Circle, Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sys
sys.path.append('/home/ambra/Desktop/cluster-morgana/')
from module_statistics import *
from statistics import mean

for index, f in enumerate(dfolders):
    print('\n' + f)
    path = directory + f + 'run0406/run0406_ID000126/csv/'
    png_path = directory + f + 'png/'
    if not os.path.isdir(png_path):
        os.mkdir(png_path)
    
    # hist params ---!
    Dwidth = 0.02
    Dnbin = int(threshold / Dwidth)

    list_ra, list_dec, list_pos = ([] for j in range(3))
    list_mode_pos, list_mean_ra, list_mean_dec, list_mean_pos = ([] for j in range(4))
    list_std_ra, list_std_dec = ([] for j in range(2))
    
    for n in range(len(texp)) :  

        # store values ---!
        filename = csvMerged[n]
        print('\n================ texp = %ds ==============\n' %texp[n])

        # load DataFrame and column names ---!
        df = pd.read_csv(path+filename)
        cols = list(df.columns)
        trials = len(df[cols[0]])
        print('* verify trials = ', trials)  
        # dropping ALL duplicte values 
        df.drop_duplicates(subset=cols[0], keep='last', inplace=False) 
        trials = len(df[cols[0]])
        print('* drop duplicate => trials = ', trials) 
        total = trials
        # drop NaN ---!
        df = df.dropna()
        trials = len(df[cols[0]])
        #df = df.reindex(np.arange(trials))
        print('* drop nan => trials = ', trials)  
        perc = 100 * (trials/total)
        print('! NaN DETECTION RATE :', 100-perc, '%')

        trial = np.array(df[cols[0]])
        raDet = np.array(df[cols[-2]])
        decDet = np.array(df[cols[-1]])

        perc = 100 * (trials/total)
        print('! DETECTION RATE :', perc, '%')

        # DETECTION ---!
        distance_det = []
        d_det = []
        ra_det = []
        dec_det = []

        for i in range(len(raDet)) :
            if -90 <= decDet[i] and decDet[i] <= 90 and 0 <= raDet[i] and raDet[i] <= 360 :
                det_coord = SkyCoord(ra = raDet[i]*u.deg, dec = decDet[i]*u.deg, frame='fk5')
                dist = float(true_coord.separation(det_coord).deg)
                distance_det.append(dist)                  

                if dist < threshold :
                    d_det.append(dist)
                    ra_det.append(raDet[i]) 
                    dec_det.append(decDet[i]) 

        perc_det = len(d_det)/total * 100
        print('! DET RATIO WITHIN %.1fdeg FROM TRUE POSITION:' %threshold, perc_det, '%')

        # MEAN DET ---!
        mean_raDet = np.nanmean(ra_det)
        std_raDet = np.nanstd(ra_det)
        mean_decDet = np.nanmean(dec_det)
        std_decDet = np.nanstd(dec_det)
        mean_posDet = np.nanmean(d_det)

        tmp = 0
        for i in range(len(d_det)) :
            tmp += d_det[i]**2

        if len(d_det) != 0.0 :
            mode_det = np.sqrt(1/(2*len(d_det)) * tmp)   
            MLE_det = 0.606/mode_det
        else:
            mode_det = np.nan
            MLE_det = np.nan
            
        print('~ ra det mean:', mean_raDet, 'ra det std:', std_raDet)
        print('~ dec det mean:', mean_decDet, 'dec det std:', std_decDet)
        print('~ pos det mean:', mean_posDet, 'pos det gamma:', mode_det)

        list_ra.append(ra_det)
        list_dec.append(dec_det)
        list_pos.append(d_det)
        list_mean_ra.append(mean_raDet)
        list_mean_dec.append(mean_decDet)
        list_mean_pos.append(mean_posDet)
        list_std_ra.append(std_raDet)
        list_std_dec.append(std_decDet)
        list_mode_pos.append(mode_det)

    # gathered per texp
#    plottage(texp, list_ra, list_dec, list_pos, list_std_ra, list_std_dec, list_mode_pos, 
#             trueRA, trueDEC, png_path)
#    print('done plotting')
print('done')


tesi_preliminary_degraded_noEBL/

================ texp = 1s ==============

* verify trials =  10000
* drop duplicate => trials =  10000
* drop nan => trials =  9978
! NaN DETECTION RATE : 0.21999999999999886 %
! DETECTION RATE : 99.78 %
! DET RATIO WITHIN 0.5deg FROM TRUE POSITION: 74.27 %
~ ra det mean: 33.05753004704624 ra det std: 0.045147058904487995
~ dec det mean: -51.84035594427709 dec det std: 0.02827552771256372
~ pos det mean: 0.03282233137789097 pos det gamma: 0.028092110366494765

================ texp = 5s ==============

* verify trials =  10000
* drop duplicate => trials =  10000
* drop nan => trials =  10000
! NaN DETECTION RATE : 0.0 %
! DETECTION RATE : 100.0 %
! DET RATIO WITHIN 0.5deg FROM TRUE POSITION: 99.99 %
~ ra det mean: 33.05713557094892 ra det std: 0.02184739434628368
~ dec det mean: -51.84076270622719 dec det std: 0.01359403632313456
~ pos det mean: 0.01709168361305904 pos det gamma: 0.01354743652408479

================ texp = 10s ==============

* veri

ParserError: Error tokenizing data. C error: Expected 6 fields in line 1503, saw 11
